In [16]:
!pip install pandas
!pip install numpy
!pip install scipy
!pip install seaborn
!pip install matplotlib
!pip install sklearn
!pip install IPython
!pip install torch
!pip install nltk


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph.
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import random

In [18]:
import pandas as pd
data = []

# Splitting the file into lines and then into fields
lines = open("train_data.txt", encoding="UTF-8").readlines()
for line in lines:
    if line.strip():  # Checking if the line is not empty
        fields = line.split(' ::: ')
        if len(fields) == 4:  # Ensuring that there are exactly 4 fields
            record = {
                'Title': fields[1],
                'Genre': fields[2],
                'Description': fields[3]
            }
            data.append(record)

# Converting the list of dictionaries to a DataFrame for better visualization
df = pd.DataFrame(data)
df.head()  # Displaying the first few records to verify the parsing


,Title,Genre,Description
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his doc...
1,Cupid (1997),thriller,A brother and sister with a past incestuous re...
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fiel...
3,The Secret Sin (1915),drama,To help their unemployed father make ends meet...
4,The Unrecovered (2007),drama,The film's title refers not only to the un-rec...


In [19]:
df1 = df[['Description', 'Genre']].copy()

In [20]:
import nltk

# Download the necessary NLTK resources
nltk.download('punkt')       # For tokenization
nltk.download('wordnet')     # For lemmatization
nltk.download('stopwords')   # For stop words

# Now you can proceed with your text processing functions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Lowercasing and removing punctuation
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenization
    tokens = word_tokenize(text)

    # Removing stop words and lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

# Apply the cleaning function to your DataFrame
df1['cleaned_description'] = df1['Description'].apply(clean_text)

# Example of cleaned text
print(df1['cleaned_description'].head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nikita\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    [listening, conversation, doctor, parent, 10ye...
1    [brother, sister, past, incestuous, relationsh...
2    [bus, empty, student, field, trip, museum, nat...
3    [help, unemployed, father, make, end, meet, ed...
4    [film, title, refers, unrecovered, body, groun...
Name: cleaned_description, dtype: object


In [21]:
texts = df1['cleaned_description'].tolist()
labels = df1['Genre'].tolist()

# Convert labels to numeric form if they are not already
label_to_ix = {label: i for i, label in enumerate(set(labels))}
numeric_labels = [label_to_ix[label] for label in labels]

# Tokenize texts
tokenized_texts = texts

# Build a vocabulary
vocab = Counter(word for sentence in tokenized_texts for word in sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [22]:
from torch.utils.data import DataLoader, Dataset

VOCAB_SIZE = len(vocab)
NUM_CLASSES = len(label_to_ix)  # Number of unique labels
EMBEDDING_DIM = 100

class TextDataset(Dataset):
    def __init__(self, texts, labels, word_to_ix):
        self.texts = texts
        self.labels = labels
        self.word_to_ix = word_to_ix

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return [self.word_to_ix[word] for word in text], label

def collate_fn(batch):
    texts, labels = zip(*batch)
    lengths = [len(text) for text in texts]
    padded_texts = torch.zeros(len(texts), max(lengths)).long()
    for i, text in enumerate(texts):
        padded_texts[i, :lengths[i]] = torch.LongTensor(text)
    return padded_texts, torch.tensor(labels)


from sklearn.model_selection import train_test_split

# Split data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    tokenized_texts, numeric_labels, test_size=0.2, random_state=42
)

train_dataset = TextDataset(train_texts, train_labels, word_to_ix)
test_dataset = TextDataset(test_texts, test_labels, word_to_ix)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

In [23]:
class CBOWClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, num_classes):
        super(CBOWClassifier, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.linear = nn.Linear(embed_size, num_classes)
        
    def forward(self, text):
        embeds = self.embeddings(text)
        embeds_mean = torch.mean(embeds, dim=1)
        out = self.linear(embeds_mean)
        return out

model = CBOWClassifier(VOCAB_SIZE, EMBEDDING_DIM, NUM_CLASSES)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [37]:
EPOCHS = 100  # Adjust as needed
def calculate_accuracy(model, data_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for context, targets in data_loader:
            outputs = model(context)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    return correct / total

for epoch in range(EPOCHS):
    total_loss = 0
    for context, target in train_loader:
        model.zero_grad()
        log_probs = model(context)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    accuracies = [calculate_accuracy(model, test_loader) for _ in range(10)]
    mean_accuracy = np.mean(accuracies)
    std_deviation = np.std(accuracies)
    print(f"Epoch {epoch}: Total Loss: {total_loss}")
    print(f"Mean Accuracy: {mean_accuracy}")
    print(f"Standard Deviation: {std_deviation}")

Epoch 0: Total Loss: 25750.02274154636
Mean Accuracy: 0.5466107165913493
Standard Deviation: 0.0007598947544079778
Epoch 1: Total Loss: 25712.236019862234
Mean Accuracy: 0.5442589689200406
Standard Deviation: 0.0013878007419331749
Epoch 2: Total Loss: 25603.445429834523
Mean Accuracy: 0.5475421931199853
Standard Deviation: 0.0013753648903297596
Epoch 3: Total Loss: 25522.788844910392
Mean Accuracy: 0.5435211657290416
Standard Deviation: 0.0014641490083899971
Epoch 4: Total Loss: 25441.90652209305
Mean Accuracy: 0.5405976205847091
Standard Deviation: 0.0015629688431955293
Epoch 5: Total Loss: 25484.65677578526
Mean Accuracy: 0.5422207876049063
Standard Deviation: 0.0007804361522840418
Epoch 6: Total Loss: 25324.33948411724
Mean Accuracy: 0.5442313013003781
Standard Deviation: 0.001195697939439069
Epoch 7: Total Loss: 25312.17762055178
Mean Accuracy: 0.535977128101079
Standard Deviation: 0.0019716401241815507
Epoch 8: Total Loss: 25209.79203722952
Mean Accuracy: 0.5460389191183251
Standa

KeyboardInterrupt: 

In [38]:
def get_predictions_and_targets(model, data_loader):
    model.eval()
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for context, targets in data_loader:
            outputs = model(context)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.tolist())
            all_targets.extend(targets.tolist())

    return all_predictions, all_targets

predictions, targets = get_predictions_and_targets(model, test_loader)


In [39]:
from sklearn.metrics import classification_report

print(classification_report(targets, predictions, target_names=label_to_ix.keys()))


              precision    recall  f1-score   support

      horror       0.56      0.61      0.58       431
     romance       0.15      0.03      0.04       151
   adventure       0.26      0.19      0.22       139
    thriller       0.24      0.15      0.18       309
       crime       0.28      0.12      0.17       107
       adult       0.46      0.38      0.41       112
        news       0.25      0.12      0.16        34
     musical       0.19      0.10      0.13        50
      family       0.25      0.17      0.20       150
   biography       0.00      0.00      0.00        61
   game-show       0.70      0.47      0.57        40
   animation       0.26      0.13      0.18       104
       short       0.35      0.38      0.36      1045
         war       0.38      0.15      0.21        20
  reality-tv       0.38      0.28      0.32       192
      action       0.34      0.25      0.29       263
 documentary       0.71      0.75      0.73      2659
      comedy       0.52    